
#	Correcting Data Classification and creating a good database
#	AUTHOR: 乔昂 - jueta
#	DATE: 13/12/2022


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_feather("Data/map_23-03-02.feather")

In [ ]:
df.head()

In [ ]:
'''
	Analysis of mapping V x FR experiments
	AUTHOR: 乔昂 - jueta
	DATE: 19/01/2023
'''

import pandas as pd
from pandas.io.json import json_normalize
import json
import matplotlib.pyplot as plt
from sklearn.utils import column_or_1d
import numpy as np
from scipy.signal import butter, lfilter



In [ ]:


colormap = []
for electro_class in df['spray_mode']:
    if electro_class == 'Intermittent':
        colormap.append('blue')
    elif electro_class == 'Cone Jet':
        colormap.append('red')
    elif electro_class == 'Dripping':
        colormap.append('green')
    elif electro_class == 'Multi Jet':
        colormap.append('purple')
    elif electro_class == 'Corona':
        colormap.append('cyan')
    elif electro_class == 'Undefined':
        colormap.append('black')
    else:
        colormap.append('black')


df.insert(1, 'colormap', colormap)


In [ ]:

df.info()
df.head()



## Using Matplotlib

In [ ]:


# df['flow rate [m3/s]'] = df['flow rate [m3/s]'].astype(float)
plt.scatter(df['flow_rate'], df['target_voltage'], color=df['colormap'])
plt.ylabel('Voltage [V]')
plt.ylim(top=9500)
plt.xlabel('Flow Rate [uL/min]')

from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Intermittent', markerfacecolor='blue', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Dripping', markerfacecolor='green', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Cone Jet', markerfacecolor='red', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Multi Jet', markerfacecolor='purple', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Undefined', markerfacecolor='black', markersize=7)]

plt.title("Automatic mapping jet modes - Pure Ethanol")
plt.legend(handles=legend_elements, bbox_to_anchor=(1.04, 1))

plt.show()


## Using Plotly express

In [ ]:
import pandas as pd
import plotly.express as px

px.scatter(df, 'flow_rate', 'target_voltage', color="spray_mode")

# ----------------------------------------------

## Mapping with undimentional Numbers

In [ ]:
import pandas as pd
df = pd.read_feather("Data/map_23-02-23.feather")

In [ ]:
electric_permitivity_of_vacum = 8.85E-12
ethanol_density = 789
eth_surface_tension = 0.02239
ehda_reynolds =	0.000003233936202
eth_viscosity = 1231.8
eth_conductivity = 0.0000554
eth_dielectric_constant = 24.3
nozzle_outer_radius = 0.00068
Q_zero = 4.54E-12
ulmin_to_m3s = 1.67E-11

In [ ]:
df.head()

In [ ]:
# Undimentional X axis
df['Undimensional_flowrate'] = df['flow_rate']*ulmin_to_m3s*ehda_reynolds/Q_zero*10000000

In [ ]:
# Undimentional Y axis
df['Undimensional_voltage'] = (df['voltage'].pow(2) * electric_permitivity_of_vacum) / (2*eth_surface_tension*nozzle_outer_radius)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

fig, ax = plt.subplots(figsize = (9, 6))
ax.scatter(df['Undimensional_flowrate'], df['Undimensional_voltage'], color=df['colormap'])

ax.set_xscale("log")
ax.set_yticks([6, 10,15 , 20, 25])
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.set_xticks([10, 20, 50,  200])
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.set_ylabel('γ1')
ax.set_xlabel('Q δμ / Q0')
ax.set_title('Mapping Experiment with Undimentional Values')


from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Intermittent', markerfacecolor='blue', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Dripping', markerfacecolor='green', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Cone Jet', markerfacecolor='red', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Multi Jet', markerfacecolor='purple', markersize=7),
                   Line2D([0], [0], marker='o', color='w', label='Undefined', markerfacecolor='black', markersize=7)]

plt.title("Automatic mapping jet modes - Pure Ethanol")
plt.legend(handles=legend_elements, bbox_to_anchor=(1.04, 1))

plt.show()